<h1>Inhaltsverzeichnis<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#D.-Systematische-Handelsstrategien" data-toc-modified-id="D.-Systematische-Handelsstrategien-1">D. Systematische Handelsstrategien</a></span></li><li><span><a href="#D.1-Ein-Asset-Strategien" data-toc-modified-id="D.1-Ein-Asset-Strategien-2">D.1 Ein-Asset-Strategien</a></span></li><li><span><a href="#D.1.1-Mean-Reversion" data-toc-modified-id="D.1.1-Mean-Reversion-3">D.1.1 Mean Reversion</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#1.-Grundlagen" data-toc-modified-id="1.-Grundlagen-3.0.1">1. Grundlagen</a></span></li><li><span><a href="#2.-Overfitting" data-toc-modified-id="2.-Overfitting-3.0.2">2. Overfitting</a></span><ul class="toc-item"><li><span><a href="#Was-ist-Overfitting?" data-toc-modified-id="Was-ist-Overfitting?-3.0.2.1">Was ist Overfitting?</a></span></li><li><span><a href="#Beispiel:-Rollierende-Fenster-(Rolling-Windows)" data-toc-modified-id="Beispiel:-Rollierende-Fenster-(Rolling-Windows)-3.0.2.2">Beispiel: Rollierende Fenster (Rolling Windows)</a></span></li><li><span><a href="#Overfitting-vermeiden" data-toc-modified-id="Overfitting-vermeiden-3.0.2.3">Overfitting vermeiden</a></span></li></ul></li><li><span><a href="#3.-Mean-Reversion-Portfolio" data-toc-modified-id="3.-Mean-Reversion-Portfolio-3.0.3">3. Mean Reversion Portfolio</a></span><ul class="toc-item"><li><span><a href="#Alternative-Ansätze" data-toc-modified-id="Alternative-Ansätze-3.0.3.1">Alternative Ansätze</a></span></li></ul></li></ul></li></ul></li></ul></div>

# D. Systematische Handelsstrategien
# D.1 Ein-Asset-Strategien
# D.1.1 Mean Reversion
### 1. Grundlagen
Mean-Reversion-Strategien gehen von der Annahme aus, dass der Preis eines Assets zufälligen Fluktuationen um einen stabilen Basistrend herum unterliegt. Daher werden Werte, die weit vom Trend (dem beobachteten Mittelwert) abweichen, dazu neigen die Richtung zu wechseln und zum Mittelwert zurückzukehren. Wenn der momentane Assetpreis ungewöhnlich hoch ist, erwarten wir, dass er wieder sinkt und steigt, wenn er ungewöhnlich niedrig ist. 

Eine Mean Reversion im Zusammenhang mit einem Aktienkurs bedeutet, dass auf Perioden, in denen der Kurs weit unter dem Mittelwert liegt Perioden folgen, in denen der Kurs ansteigt und umgekehrt. Wir können dies ausnutzen, indem wir die Aktie kaufen (d.h., long gehen), wenn der Preis niedriger als erwartet ist und verkaufen (d.h., short gehen), wenn der Preis höher als erwartet ist. Lassen Sie uns die Idee am Beispiel der Aktie Procter & Gamble (Ticker: PG) verdeutlichen. Zunächst plotten wir den Aktienkurs zusammen mit seinem rollierenden Mittelwert, um zu sehen, ob der Kurs zum Mittelwert zurückkehrt.

In [1]:
#pip install pandas-datareader

In [2]:
# Import necessary libaries
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader.data as web
from datetime import datetime

# Load the prices data for a stock
start = '2013-06-01'
end = '2016-12-31'
m= 'PG'
prices = web.get_data_yahoo(m, start, end)['Adj Close']
prices = pd.DataFrame(prices)

# Compute the cumulative moving average of the price
prices['mu'] = [prices['Adj Close'][:i].mean() for i in range(len(prices))]
# Plot the price and the moving average
plt.figure(figsize=(15,7))
plt.plot(prices['Adj Close'])
plt.plot(prices['mu'])
plt.show()

TypeError: string indices must be integers

Da wir den rollierenden Durchschnitt berechnen, ist zu beachten, dass die "Rückkehr zum Mittelwert" nicht unbedingt bedeutet, dass die bisherigen Höchst- und Niedrigwerte wieder erreicht werden.

Um anhand dieser Strategie handeln zu können, müssen wir quantifizieren was es bedeutet, wenn der Preis höher oder niedriger als erwartet ausfällt. Hierfür ist es nützlich, den z-Score des Preises an jedem Tag zu berechnen, der uns sagt wie viele Standardabweichungen vom Mittelwert der aktuelle Aktienkurs entfernt ist:

$$ z = \frac{x - \mu}{\sigma} $$

wobei $x$ der aktuelle Kurs, $\mu$ der Stichprobenmittelwert des Kurses und $\sigma$ seine Standardabweichung ist. Ein Preis mit einem z-Score $> 1$ liegt somit um mehr als eine Standardabweichung über dem Mittelwert. Dies ist ein mögliches Signal für einen Leerverkauf (Short Sale) der Aktie. Wenn der Preis an einem Tag einen z-Score $< -1$ hat, ist dies umgekehrt ein Signal für einen Aktienkauf (eine Long Position). Liegt der Preis innerhalb einer halben Standardabweichung um den Mittelwert, werden wir unsere Position glattstellen. Implementieren wir nun diese Strategie:

In [ ]:
# Compute the z-scores for each day using the historical data up to that day
zscores = [(prices['Adj Close'][i] - prices['mu'][i]) / np.std(prices['Adj Close'][:i]) for i in range(len(prices))]

# Start with no money and no positions
money = 0
count = 0
for i in range(len(prices)):
    # Sell short if the z-score is > 1
    if zscores[i] > 1:
        money += prices['Adj Close'][i]
        count -= 1
    # Buy long if the z-score is < -1
    elif zscores[i] < -1:
        money -= prices['Adj Close'][i]
        count += 1
    # Clear positions if the z-score between -.5 and .5
    elif abs(zscores[i]) < 0.5:
        money += count*prices['Adj Close'][i]
        count = 0
print(money)

### 2. Overfitting

#### Was ist Overfitting?

Jeder systematischen, regelgebundenen Handelsstrategie liegt ein Modell zugrunde, welches die Portfoliogewichte und Handelssignale liefert. Beim Konstruieren eines solchen Modells stimmen wir sowohl die Parameter als auch das Modell als solches ab, indem wir es mit Hilfe von Beispieldaten anpassen (d.h., parametrisieren). Dann verwenden wir das Modell und die Parameter, um zukünftige Daten (und damit Handelssignale) vorherzusagen, die wir noch nicht beobachtet haben. Ein Modell wird als *overfitted* bezeichnet, wenn es übermäßig empfindlich auf Noise (d.h., Zufallsrauschen - das Gegenteil von Information) in den Stichprobendaten reagiert und daher den zugrundeliegenden Datengenerierungs-Prozess nicht zutreffend widerspiegelt.

Um zu verstehen, warum dies geschieht, muss man die in jedem Datensatz vorhandene Menge an Noise berücksichtigen. Man kann einen Datensatz betrachten als die Summe von Information ($D_{T}$), d.h., die wahren zugrundeliegenden Daten, die aus dem Prozess stammen, den wir zu modellieren versuchen, und Zufallsrauschen ($\epsilon$). Was wir also tatsächlich sehen, ist $D = D_{T} + \epsilon$. Durch den Aufbau eines komplexen Modells könnten wir es theoretisch so anpassen, dass es nicht nur die Information "lernt", sondern auch das Datenrauschen $\epsilon$. Wenn wir dieses Modell jedoch an zukünftigen Daten aus einer neuen (Out-of-Sample) Stichprobe testen, wird es $D_{T}$ nicht mit großer Genauigkeit vorhersagen können.

Im Kern ist für uns eine Parametrisierung unseres Modells an eine Stichprobe nur deshalb wichtig, um damit möglichst genau zukünftige Daten vorhersagen zu können. Wir wollen also nur die Information aus den Daten lernen, und nicht das zufällige Rauschen! Die beiden Hauptursachen für Overfitting sind:
* Eine zu kleine Stichprobe, so dass Rauschen und Trend (Information) nicht unterscheidbar sind;
* Die Auswahl eines übermäßig komplexen Modells, das den Datensatz perfekt auswendig lernt (fitted);

Wenn man dies auf Aktien anwendet, so ist ein Modell, das viele spezifische Regeln auf der Grundlage bestimmter vergangener Ereignisse erlernt, fast immer overfitted. Aus diesem Grund ist das Black-Box Machine Learning (Entscheidungsbäume, Neuronale Netze, usw.) so gefährlich, wenn es nicht korrekt durchgeführt wird.

#### Beispiel: Rollierende Fenster (Rolling Windows)

Eine der Herausforderungen bei der Erstellung eines Modells welches rollierende Parameterschätzungen verwendet, wie z.B. rollierende Mittelwerte oder rollierende Beta-Werte, ist die Wahl der Länge des Fensters (Window Length). Ein längeres Fenster wird langfristige Trends berücksichtigen und weniger volatil sein, aber es wird auch langsamer auf neue Beobachtungen reagieren. Die Wahl der Window Length wirkt sich stark auf die Schätzung der rollierenden Parameter aus und kann die Art und Weise verändern, wie wir die Daten sehen und behandeln. Zur Verdeutlichung  berechnen wir im Folgenden die rollierenden Durchschnitte eines Aktienpreises für verschiedene Window Lengths:

In [ ]:
# Load the pricing data for a stock
start = '2012-01-01'
end = '2014-06-30'
assets = ['MCD']

prices = web.get_data_yahoo(assets, start, end)['Adj Close']
prices = pd.DataFrame(prices)

asset = prices.iloc[:, 0]
# Compute rolling averages for various window lengths
mu_30d = asset.rolling(window=30, center=False).mean()
mu_60d = asset.rolling(window=60, center=False).mean()
mu_100d = asset.rolling(window=100, center=False).mean()

# Plot asset pricing data with rolling means from the 100th day, when all the means become available
plt.figure(figsize=(15,7))
plt.plot(asset[100:], label='Asset')
plt.plot(mu_30d[100:], label='30d MA')
plt.plot(mu_60d[100:], label='60d MA')
plt.plot(mu_100d[100:], label='100d MA')
plt.xlabel('Day')
plt.ylabel('Price')
plt.legend()
plt.show()

Wenn wir die Länge danach wählen wie gut unser Modell oder unser Algorithmus funktioniert, dann betreiben wir Overfitting. Nachfolgend implementieren wir einen einfachen Handelsalgorithmus, der darauf wettet, dass der Aktienkurs zum rollierenden Mittelwert zurückkehrt. Wir verwenden die Performance dieses Algorithmus, um die optimale (d.h., die Performance-maximierende) Window Length zu ermitteln. Wenn wir jedoch eine andere Analyseperiode in Betracht ziehen, ist diese Window Length mit großer Wahrscheinlichkeit nicht mehr optimal. Das liegt daran, dass unsere ursprüngliche Wahl an den Beispieldaten "overfitted" wurde.

In [ ]:
# Trade using a simple mean-reversion strategy
def trade(stock, length):
    
    # If window length is 0, algorithm doesn't make sense, so exit
    if length == 0:
        return 0
    
    # Compute rolling mean and rolling standard deviation
    mu = stock.rolling(window=length, center=False).mean()
    std = stock.rolling(window=length, center=False).std()
    
    # Compute the z-scores for each day using the historical data up to that day
    zscores = (stock - mu)/std
    
    # Simulate trading
    # Start with no money and no positions
    money = 0
    count = 0
    for i in range(len(stock)):
        # Sell short if the z-score is > 1
        if zscores[i] > 1:
            money += stock[i]
            count -= 1
        # Buy long if the z-score is < -1
        elif zscores[i] < -1:
            money -= stock[i]
            count += 1
        # Clear positions if the z-score between -.5 and .5
        elif abs(zscores[i]) < 0.5:
            money += count*stock[i]
            count = 0
    return money

In [ ]:
# Find the window length 0-254 that gives the highest returns using this strategy
length_scores = [trade(asset, l) for l in range(255)]
best_length = np.argmax(length_scores)
print('Best window length:', best_length)

In [ ]:
# Get pricing data for a different timeframe
start2 = '2014-06-30'
end2 = '2017-01-01'
assets = ['MCD']

prices2 = web.get_data_yahoo(assets, start2, end2)['Adj Close']
prices2 = pd.DataFrame(prices2)
asset2 = prices2.iloc[:, 0]

# Find the returns during this period using what we think is the best window length
length_scores2 = [trade(asset2, l) for l in range(255)]
print(best_length, 'day window:', length_scores2[best_length])

# Find the best window length based on this dataset, and the returns using this window length
best_length2 = np.argmax(length_scores2)
print(best_length2, 'day window:', length_scores2[best_length2])

Es wird deutlich, dass eine Anpassung an unsere Beispieldaten nicht immer zu guten Ergebnissen in der Zukunft führt. Lassen Sie uns für beide Zeitperioden den Zusammenhang zwischen Performance und Window Length graphisch darstellen:

In [ ]:
plt.figure(figsize=(15,7))
plt.plot(length_scores)
plt.plot(length_scores2)
plt.xlabel('Window length')
plt.ylabel('Score')
plt.legend(['2012-2014', '2014-2016'])
plt.show()

#### Overfitting vermeiden

Wir können versuchen Overfitting zu vermeiden, indem wir große Stichproben verwenden, vernünftige und einfache Modelle wählen und nicht die Parameterwerte herauspicken, die augenscheinlich am besten zu den Daten passen. Jedoch ist anzumerken, dass schon zwei Backtests zu einem Overfitting führen.

##### - Out-of-Sample Testing

Um sicherzugehen, dass wir unser Modell nicht durch Overfitting unbrauchbar machen, müssen wir es Out-of-Sample testen. Das heißt, wir müssen Daten sammeln, die wir bei der Konstruktion des Modells nicht verwendet haben, und testen, ob unser Modell weiterhin funktioniert. Wenn wir nicht nach Belieben große Mengen an zusätzlichen Daten sammeln können, sollten wir unsere Stichprobe in zwei Teilmengen separieren, von denen eine nur für das Out-of-Sample Testing reserviert wird.

##### - Häufiger Fehler: Missbrauch von Out-of-Sample Daten

Nehmen wir die folgende Situation an. Sie konstruieren ein Modell auf In-Sample Daten, testen es Out-of-Sample und kommen zu dem Schluss, dass es nicht funktioniert. Dann wiederholen Sie diesen Prozess, bis Sie ein Modell finden, das funktioniert. Bei diesem Vorgehen handelt es sich immer noch um Overfitting, da Sie das Modell an die Out-of-Sample Daten angepasst haben, indem sie diese Daten wiederholt verwendet haben, und wenn Sie tatsächlich mit echten Daten außerhalb der Stichprobe testen, wird Ihr Modell wahrscheinlich schlecht abschneiden.


##### - Kreuzvalidierung

Bei der Kreuzvalidierung (Cross Validation) werden die Daten in n Teilmengen zerlegt. Die optimalen Parameter werden für n-1 kombinierte Teilmengen geschätzt und an der zurückgehaltenen Teilmenge getestet. Indem wir diesen Prozeß n-mal durchführen, ein Mal für jede der n zurückgehaltenen Teilmengen, können wir feststellen, wie stabil unsere Parameterschätzungen sind und wie viel Vorhersagekraft sie für Daten haben, die nicht aus dem ursprünglichen Datensatz stammen.

### 3. Mean Reversion Portfolio

Die Gefahr der Anwendung von Mean Reversion Strategien auf eine einzelne Aktie besteht darin, dass wir uns unter anderem den allgemeinen Bewegungen des Marktes und dem Erfolg oder Misserfolg eines einzelnen Unternehmens aussetzen. Wenn es einen anhaltenden Trend gibt, der sich auf den Kurs des Wertpapiers auswirkt, werden wir den Vermögenswert immer wieder unterbewerten (wenn sich der Kurs stetig nach oben bewegt) oder überbewerten (wenn der Kurs fällt). Nun lernen wir eine Strategie kennen, die dieses Risiko mindert.

Anstatt den Mittelwert der historischen Renditen eines Vermögenswertes zu nehmen, können wir uns den Mittelwert der Renditen aller Aktien, z.B. des S&P 500 ansehen. Unter der Hypothese, dass die Aktien mit der schlechtesten Performance im vergangenen Zeitraum im aktuellen Zeitraum besser abschneiden werden (d.h., sie sind wahrscheinlich unterbewertet) und umgekehrt, setzen wir bei Aktien, die sich schlecht entwickelt haben, auf steigende Kurse und bei Aktien, die sich gut entwickelt haben, auf fallende Kurse.

Dieser Ansatz hat den Vorteil, dass er marktneutral ist, so dass wir Aktien nicht als unterbewertet behandeln, nur weil der Markt als Ganzes fällt, oder als überbewertet, wenn der Markt steigt. Darüber hinaus werden wir durch die Aufnahme einer großen Anzahl von Wertpapieren in unser Portfolio wahrscheinlich auf viele Fälle stoßen, in denen unsere Vorhersage zutrifft.

Um ein Portfolio zu konstruieren, das sich die Mean Reversion zunutze macht, wählen wir zunächst ein Universum aus, z.B. alle Aktien des S&P 500 oder die an der NYSE am meisten gehandelten Aktien. Auf Basis dieses Universums schichten wir unser Portfolio in jeder Periode (z.B. jede Woche) neu um (Rebalancing), indem wir bei den 20% der Aktien mit der geringsten Rendite (Past Loser Stocks) über die vergangene Periode Long gehen und die obersten 20% der Aktien mit der höchsten Rendite (Past Winner) shorten. Befindet sich eine Aktie des Universums in keinem dieser Quintile, nehmen wir sie nicht in unser Portfolio auf.

Lassen Sie uns ein Beispiel mit nur 10 Aktien konstruieren:

In [ ]:
# Fetch prices data for 10 stocks from different sectors and plot returns
start = '2016-12-01'
end = '2016-12-31'
assets = ['AAPL', 'AIG', 'C', 'T', 'PG', 'JNJ', 'EOG', 'DD', 'MET', 'AMGN']

prices_list = []
for ticker in assets:
    try:
        prices = web.get_data_yahoo(ticker, start, end)['Adj Close']
        prices = pd.DataFrame(prices)
        prices.columns = [ticker]
        prices_list.append(prices)
    except:
        pass
    prices_df = pd.concat(prices_list,axis=1)

returns = prices_df/prices_df.shift(-1) -1
returns.plot(figsize=(15,7), color=['r', 'g', 'b', 'k', 'c', 'm', 'orange', 'chartreuse', 'slateblue', 'silver'])
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.ylabel('Returns')

# Convert to numpy array to make manipulation easier
data = np.array(prices_df)


Wir gehen davon aus, dass die Aktien, die sich in der ersten Woche eines Monats gut entwickeln, im restlichen Verlauf des Monat schlechter performen werden, während diejenigen, die sich anfangs schlecht entwickeln, an Wert gewinnen werden.

In [ ]:
# For each security, take the return for the first week
wreturns = (data[4] - data[0])/data[0]
# Rank securities by return, with 0 being the lowest return
order = wreturns.argsort()
ranks = order.argsort()

# For each security, take the return for the month following the first week
# Normalization for the time period doesn't matter since we're only using the returns to rank them
mreturns = (data[-1] - data[5])/data[5]
order2 = mreturns.argsort()
ranks2 = order2.argsort()

# Plot the returns for the first week vs returns for the next month to visualize them
plt.figure(figsize=(15,7))
plt.scatter(wreturns, mreturns)
plt.xlabel('Returns for the first week')
plt.ylabel('Returns for the following month')

Die Renditen sehen aus als wären sie unkorreliert, aber was wäre passiert, wenn wir bei der Untersuchung der Renditen der vergangenen Woche die Strategie der Mean Reversion verfolgt hätten?

In [ ]:
ranks

In [ ]:
longs = np.array([int(x < 2)for x in ranks])
longs
shorts

In [ ]:
# Go long (by one share each) in the bottom 20% of securities and short in the top 20%
longs = np.array([int(x < 2)for x in ranks])
shorts = np.array([int(x > 7) for x in ranks])
print('Going long in:', [assets[i] for i in range(len(assets)) if longs[i]])
print('Going short in:', [assets[i] for i in range(len(assets)) if shorts[i]])

# Resolve all positions and calculate how much we would have earned
print('Yield:', sum((data[-1] - data[4])*(longs - shorts)))

#### Alternative Ansätze

Mean Reversion Strategien gehen davon aus, dass sich die Preis- und/oder Renditetrends tendenziell umkehren. Am entgegengesetzten Ende des Spektrums stehen trendfolgende oder momentumbasierte Strategien. Diese gehen davon aus, dass sich die Preise trotz Schwankungen zumindest temporär weiter in die Richtung bewegen, in die sie sich zuvor bewegt haben. Diese werden in Abschnitt D.1.2 ausführlicher behandelt.